In [ ]:
import spotipy
from spotipy.oauth2 import SpotifyOAuth
from spotipy.exceptions import SpotifyException
import jwt

# Initialize SpotifyOAuth with your credentials

#this is where the user would need to enter their own credentials
sp_oauth = SpotifyOAuth(
    client_id = '73ab5349d4284b759c07843e5d202eff',
    client_secret = '6f02411e145b441dac805c627a2d90d4',
    redirect_uri='http://localhost:8881/callback',
    scope='playlist-read-private playlist-read-collaborative user-library-read'
)

sp = spotipy.Spotify(auth_manager=sp_oauth)

# so this is a function to list all of the user's playlists
#playlists that the user either created or followed
def list_user_playlists():
    playlist = sp.current_user_playlists()
    #can also put playlist in playlist["item"]
    for playlist in playlist["items"]:
        print(f"Playlist Name: {playlist['name']}, Playlist ID: {playlist['id']}")

list_user_playlists()
